# Install

In [1]:
!pip install -U transformers simpletransformers

     |████████████████████████████████| 3.1 MB 7.1 MB/s 
     |████████████████████████████████| 231 kB 37.4 MB/s 
     |████████████████████████████████| 895 kB 36.2 MB/s 
     |████████████████████████████████| 596 kB 19.2 MB/s 
     |████████████████████████████████| 59 kB 3.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.0 MB/s 
     |████████████████████████████████| 8.3 MB 25.9 MB/s 
     |████████████████████████████████| 1.2 MB 41.2 MB/s 
     |████████████████████████████████| 1.7 MB 40.9 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 290 kB 51.3 MB/s 
     |████████████████████████████████| 139 kB 56.0 MB/s 
     |████████████████████████████████| 180 kB 56.4 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 132 kB 54.8 MB/s 
     |████████████████████████████████| 243 kB 41.9 MB/s 
     |█████████████████

In [2]:
!pip install tweet-preprocessor

In [3]:
pip install tensorboardX

     |████████████████████████████████| 124 kB 7.2 MB/s 


# Load data
- Load data
- Split train - test dataset
- X_train, y_train, X_test, y_test and one hot encoding

In [4]:
import os
import pandas as pd
import numpy as np
import string
import re
import math
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics

#handling text data
from torchtext import data
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import nltk 
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import files
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from preprocessor.api import clean

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

DATA = "https://github.com/socd06/private_nlp/raw/master/data/mtsamples.csv"

filtered_labels = [
    "Surgery",
    "Consult - History and Phy.",
    "Cardiovascular / Pulmonary",
    "Orthopedic",
]
data = pd.read_csv(DATA, usecols=['medical_specialty', 'transcription']).dropna()
data.columns = ['labels', 'text']
data['labels'] = [i.strip() if (i.strip() in filtered_labels) else 'Other' for i in data.labels.to_list()]

train, test = train_test_split(data, test_size=0.4, stratify=data.labels, random_state=0)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
X_train = train['text']
y_train = train['labels']

X_test = test['text']
y_test = test['labels']

y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

ohe2 = OneHotEncoder()
y_train = ohe2.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = ohe2.transform(y_test.reshape(-1, 1)).toarray()

# Preprocessing
- Lower case
- Remove punctuation, alphanumerics
- Lemmatization
- Remove stop words (stopword.txt)
- Join words back together


In [6]:
print(train)

                          labels                                               text
0     Consult - History and Phy.  REVIEW OF SYSTEMS,GENERAL:  Negative weakness,...
1                        Surgery  PREOPERATIVE DIAGNOSIS: , Herniated nucleus pu...
2                          Other  PREOPERATIVE DIAGNOSIS:  Large juxtarenal abdo...
3     Consult - History and Phy.  GENERAL:,  Negative weakness, negative fatigue...
4                          Other  The patient underwent an overnight polysomnogr...
...                          ...                                                ...
2974                       Other  ADMISSION DIAGNOSES:,1.  Severe menometrorrhag...
2975                  Orthopedic  TITLE OF OPERATION: , Intramedullary nail fixa...
2976                       Other  PREOPERATIVE DIAGNOSES:  ,1.  Large herniated ...
2977                       Other  ALLOWED CONDITIONS:,  Sprain of left knee and ...
2978  Cardiovascular / Pulmonary  HISTORY AND PHYSICAL:  ,The patient is a 1

In [7]:
pre_clean = lambda x: clean(x)
remove_nchar =  lambda x: x.replace('\n',' ')
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

In [8]:
X_train = X_train.map(pre_clean).map(remove_nchar).map(alphanumeric).map(punc_lower)
X_test = X_test.map(pre_clean).map(remove_nchar).map(alphanumeric).map(punc_lower)

In [9]:
# Create a text lemmatisation function
def lemma(text):
    lemma = nltk.wordnet.WordNetLemmatizer()
    return lemma.lemmatize(text, pos='v')

In [10]:
tokenizer = TreebankWordTokenizer()

In [11]:
# Create a text tokenizer function
def token_lem(text):
    tokenizer = TreebankWordTokenizer()
    return [lemma(x) for x in tokenizer.tokenize(text)]

In [12]:
import nltk
nltk.download('wordnet')
token_stem = lambda x: token_stem(x)
X_train = X_train.apply(token_lem)
X_test = X_test.apply(token_lem)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [13]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab/Talosix")

Mounted at /content/drive


In [14]:
medical_stop_words = open('stopwords.txt', 'r')
medical_stop_words = medical_stop_words.readlines()
medical_stop_words = medical_stop_words[9:]
medical_stop_words = pd.Series(medical_stop_words)
medical_stop_words = medical_stop_words.map(pre_clean).map(remove_nchar).map(alphanumeric).map(punc_lower)

def remove_last_space(word):
    word = " ".join(word.split())
    return word
medical_stop_word = medical_stop_words.apply(lambda x: remove_last_space(x))
medical_stop_words = pd.Series.tolist(medical_stop_words)


import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(tokenized_list):
    tmp = [word for word in tokenized_list if word not in stopwords]
    text = [word for word in tmp if word not in medical_stop_words]
    return text

X_train = X_train.apply(lambda x: remove_stopwords(x))
X_test = X_test.apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
# Join words back together so they can be passed to CountVectorizer
X_train = X_train.apply(lambda x: ' '.join(x))
X_test = X_test.apply(lambda x: ' '.join(x))

# Compute TF-IDF


In [16]:
# building TFIDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features= 10000)
X_train_2 = vectorizer.fit_transform(X_train).toarray().astype('float32')
X_test_2 = vectorizer.transform(X_test).toarray().astype('float32')

In [17]:
print(X_test_2.shape)

(1987, 10000)


# Smote

In [18]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train_2, y_train = oversample.fit_resample(X_train_2, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [19]:
X_train_2.shape

(7905, 10000)